In [2]:
import numpy as np
import pandas as pd

In [5]:
solar = pd.read_csv("datasets/solar.txt", sep = ',', header = None)
solar.shape

(52560, 137)

In [7]:
electricity = pd.read_csv("datasets/electricity.txt", sep = ',', header = None)
electricity.shape

(26304, 321)

In [8]:
traffic = pd.read_csv("datasets/traffic.txt", sep = ',', header = None)
traffic.shape

(17544, 862)

In [38]:
exchange_rate = pd.read_csv("datasets/exchange_rate.txt", sep = ',', header = None)
exchange_rate.shape

(7588, 8)

In [115]:
exchange_rate.describe()

,0,1,2,3,4,5,6,7
count,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000,7588.000000
mean,0.776974,1.634105,0.821811,0.848146,0.142833,0.009343,0.654418,0.669673
std,0.136620,0.161544,0.117123,0.168874,0.023996,0.001458,0.115292,0.082836
min,0.483297,1.211534,0.618582,0.548617,0.109292,0.006254,0.393153,0.523834
25%,0.701422,1.532887,0.727901,0.696864,0.120814,0.008331,0.566000,0.593287
50%,0.761377,1.606574,0.811582,0.813959,0.145212,0.009151,0.669187,0.662767
75%,0.873477,1.707646,0.920394,1.014018,0.159948,0.009995,0.734901,0.731835
max,1.102536,2.109000,1.091524,1.374079,0.237954,0.013202,0.882379,0.832556


In [114]:
exchange_rate_np_array = exchange_rate.to_numpy()
np.save('exchange_rate.npy', exchange_rate_np_array)

In [39]:
# https://analyticsindiamag.com/complete-guide-to-dickey-fuller-test-in-time-series-analysis/

from statsmodels.tsa.stattools import adfuller

def stationarity_check(series):
    result = adfuller(series, autolag='AIC')

    print(column)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))
    if result[0] < result[4]["5%"]:
        print ("Reject Ho - Time Series is Stationary")
        return True
    else:
        print ("Failed to Reject Ho - Time Series is Non-Stationary")
        return False

In [40]:
df = exchange_rate

for column in df:
    series = df[column].values
    is_stationary = stationarity_check(series)

0
ADF Statistic: -1.664994
p-value: 0.449233
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Failed to Reject Ho - Time Series is Non-Stationary
1
ADF Statistic: -2.149718
p-value: 0.224998
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Failed to Reject Ho - Time Series is Non-Stationary
2
ADF Statistic: -1.352572
p-value: 0.604791
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Failed to Reject Ho - Time Series is Non-Stationary
3
ADF Statistic: -1.586708
p-value: 0.490267
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Failed to Reject Ho - Time Series is Non-Stationary
4
ADF Statistic: -2.869174
p-value: 0.049052
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
5
ADF Statistic: -2.120121
p-value: 0.236501
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Failed to Reject Ho - Time Series is Non-Stationary
6
ADF Statistic: -1.728197
p-value: 0.416646
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Failed to

In [42]:
# taking 1st order difference to make the time series stationary

df_diff = pd.DataFrame()
for column in df:
    series = df[column]
    series_diff = series.diff()
    df_diff[column] = series_diff.dropna()

In [43]:
for column in df_diff:
    series = df_diff[column].values
    is_stationary = stationarity_check(series)

0
ADF Statistic: -99.393431
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
1
ADF Statistic: -22.636334
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
2
ADF Statistic: -35.296871
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
3
ADF Statistic: -24.489297
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
4
ADF Statistic: -47.260145
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
5
ADF Statistic: -52.537349
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
6
ADF Statistic: -52.335680
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary
7
ADF Statistic: -20.503845
p-valu

In [52]:
# using auto arima to try to come up with possible good values for p and q parameters

from pmdarima import auto_arima

pq = []
for column in df:
    print(column)
    model = auto_arima(df[column], test='adf', stepwise=True, trace=True)
    parameters = model.get_params().get('order')
    pq.append(parameters)

print(pq)

0
Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-56098.154, Time=2.50 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-55972.211, Time=1.68 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-56102.338, Time=0.81 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-56100.772, Time=3.89 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-55974.195, Time=0.33 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-56100.340, Time=1.48 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-56100.341, Time=1.73 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-56098.328, Time=3.52 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=-56104.319, Time=0.42 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=-56102.321, Time=0.41 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=-56102.322, Time=1.23 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=-56102.753, Time=0.61 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=-56100.316, Time=2.26 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0]          
Total fit time: 20.886 s

In [64]:
only_pq = []
for values in pq:
    pq_tuple = (values[0], values[2])
    only_pq.append(pq_tuple)
only_pq = [t for t in (set(tuple(i) for i in only_pq))]
print(only_pq)

[(1, 0), (0, 3), (2, 2), (0, 1)]


In [48]:
# train = df_diff.sample(frac = 0.90)
# test = df_diff.drop(train.index)
# print(train.shape)
# print(test.shape)

(6828, 8)
(759, 8)


In [65]:
train = df[0:-15]
test = df[-15:]
train_diff = df_diff[0:-15]
test_diff = df_diff[-15:]

In [66]:
# https://colab.research.google.com/github/Apress/hands-on-time-series-analylsis-python/blob/master/Chapter%204/7.%20VARMA%20with%20Auto%20Arima.ipynb#scrollTo=ax7ukNmu-TsX
# https://analyticsindiamag.com/complete-guide-to-dickey-fuller-test-in-time-series-analysis/


def inverse_diff(actual_df, pred_df):
    df_res = pred_df.copy()
    columns = actual_df.columns
    for col in columns:
        df_res[str(col)+'_1st_inv_diff'] = actual_df[col].iloc[-1] + df_res[col].cumsum()
    return df_res

In [67]:
# fining best value of p and q parameters now for VARMA

from sklearn import metrics
from statsmodels.tsa.statespace.varmax import VARMAX

grid_search_results_columns = ['p', 'q']
for column in df:
    grid_search_results_columns.append('rmse ' + str(column))
df_results_grid_search = pd.DataFrame(columns=grid_search_results_columns)

for i in only_pq:
    model = VARMAX(train_diff, order=(i[0],i[1])).fit()
    predicted = model.forecast(steps=15)
    inv_diff_predicted = inverse_diff(df, predicted)
    row = {'p': i[0], 'q': i[1]}
    for column in df:
        rmse = np.sqrt(metrics.mean_squared_error(test[column], inv_diff_predicted[str(column) + '_1st_inv_diff']))
        row['rmse ' + str(column)] = rmse
    df_results_grid_search = df_results_grid_search.append(row, ignore_index=True)

df_results_grid_search.sort_values(by=grid_search_results_columns[2:])

C:\Users\mehta\anaconda3\envs\aidi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mehta\anaconda3\envs\aidi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\mehta\AppData\Local\Temp\ipykernel_13288\615246142.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_results_grid_search = df_results_grid_search.append(row, ignore_index=True)
C:\Users\mehta\anaconda3\envs\aidi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mehta\anaconda3\envs\aidi\lib\sit

,p,q,rmse 0,rmse 1,rmse 2,rmse 3,rmse 4,rmse 5,rmse 6,rmse 7
2,2.0,2.0,0.003400,0.007304,0.004110,0.006103,0.000159,0.000042,0.003393,0.001048
3,0.0,1.0,0.003437,0.007135,0.004139,0.006346,0.000128,0.000043,0.003212,0.001037
1,0.0,3.0,0.003440,0.007545,0.004208,0.006485,0.000208,0.000044,0.003530,0.001061
0,1.0,0.0,0.003445,0.007274,0.004190,0.006350,0.000127,0.000043,0.003273,0.001038


In [76]:
df_diff.shape

(7587, 8)

In [111]:
test_size = df_diff.shape[0] - round(df_diff.shape[0] * 90 / 100)
print(test_size)

759


In [81]:
train_diff = df_diff[0:-test_size]

In [82]:
# training my VARMA model on 90% of the dataset

model = VARMAX(train_diff, order=(2,2)).fit(disp=False)

C:\Users\mehta\anaconda3\envs\aidi\lib\site-packages\statsmodels\tsa\statespace\varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Users\mehta\anaconda3\envs\aidi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mehta\anaconda3\envs\aidi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [108]:
result = model.forecast(steps=test_size)

C:\Users\mehta\anaconda3\envs\aidi\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [109]:
inv_diff_result = inverse_diff(df, result)
print(inv_diff_result)

                0            1            2            3            4  \
6828 -1457.767881  -698.551140 -2094.523485 -2055.776299  5608.274447   
6829 -1029.454084 -1814.151509  -952.694336 -1456.052171 -3494.323611   
6830   -47.777742    44.813254   341.339858   -39.508018  3872.271813   
6831   -30.904484    49.595965  -195.232001   -66.619514  1322.461398   
6832   -55.163219     6.928549    55.870638    60.611975  1151.644264   
...           ...          ...          ...          ...          ...   
7582    -6.127403    -1.692079    -7.372311    -5.339305   -61.922452   
7583    -6.127403    -1.692079    -7.372311    -5.339305   -61.922452   
7584    -6.127403    -1.692079    -7.372311    -5.339305   -61.922452   
7585    -6.127403    -1.692079    -7.372311    -5.339305   -61.922452   
7586    -6.127403    -1.692079    -7.372311    -5.339305   -61.922452   

                 5            6            7  0_1st_inv_diff  1_1st_inv_diff  \
6828 -3.467611e+05  -493.769261  8291.49403

In [113]:
# https://www.marinedatascience.co/blog/2019/01/07/normalizing-the-rmse/

test = df[-test_size:]

for column in df:
    rmse = np.sqrt(metrics.mean_squared_error(test[column], inv_diff_result[str(column) + '_1st_inv_diff']))
    print('RMSE for {}: {:.3f}'.format(column, rmse))
    nrmse = rmse / (test[column].max() - test[column].min())
    print('NRMSE (maxmin) for {}: {:.3f}'.format(column, nrmse))
    nrmse_mean = rmse / (test[column].mean())
    print('NRMSE (mean) for {}: {:.3f}'.format(column, nrmse_mean))

RMSE for 0: 5102.273
NRMSE (maxmin) for 0: 30810.084
NRMSE (mean) for 0: 6795.561
RMSE for 1: 3065.118
NRMSE (maxmin) for 1: 8134.041
NRMSE (mean) for 1: 2119.498
RMSE for 2: 5824.894
NRMSE (maxmin) for 2: 29850.124
NRMSE (mean) for 2: 7537.557
RMSE for 3: 5694.143
NRMSE (maxmin) for 3: 27394.786
NRMSE (mean) for 3: 5540.286
RMSE for 4: 19274.351
NRMSE (maxmin) for 4: 1018136.983
NRMSE (mean) for 4: 124262.808
RMSE for 5: 12594753.165
NRMSE (maxmin) for 5: 6131817509.901
NRMSE (mean) for 5: 1443150325.263
RMSE for 6: 8302.924
NRMSE (maxmin) for 6: 50632.215
NRMSE (mean) for 6: 11835.776
RMSE for 7: 6488.931
NRMSE (maxmin) for 7: 83360.284
NRMSE (mean) for 7: 8921.813


In [107]:
import pickle as pkl

filename = 'var_model.pickle'
pkl.dump(model, open(filename, 'wb'))